# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.4660e18,"""2855644311015502179""","""../../../../data/SampleHDS.jso…","""15988102914565198687""","""14569942462415761804""","""17702028769948025610""",false
6.6351e18,"""6653280497393247691""","""../../../../data/SampleHDS.jso…","""2150718323852079883""","""14569942462415761804""","""2402782469975081286""",true
5.6655e18,"""7304843016400893670""","""../../../../data/SampleHDS.jso…","""1924447732341923694""","""9013652043069145205""","""17702028769948025610""",false
1.8116e19,"""6653280497393247691""","""../../../../data/SampleHDS.jso…","""1137598219016281252""","""12199825978631944732""","""17702028769948025610""",false
2.8683e18,"""2855644311015502179""","""../../../../data/SampleHDS.jso…","""2647444276220279202""","""12199825978631944732""","""17117036990582136667""",true
1.0913e19,"""7304843016400893670""","""../../../../data/SampleHDS.jso…","""17631971037695221257""","""17551784763537844255""",null,false
1.7788e19,"""7304843016400893670""","""../../../../data/SampleHDS.jso…","""9781661880198203214""","""12199825978631944732""","""17117036990582136667""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
9.2544e18,"""5406922016267987652""","""../../../../data/SampleHDS.jso…","""8695335668899058889""","""FirstMortgage30yr""","""13667322956089985814""","""Rejected"""
1.4412e19,"""4005774257469024620""","""../../../../data/SampleHDS.jso…","""11483706693137431102""","""FirstMortgage30yr""","""2434710285271227197""","""Accepted"""
1.7351e19,"""8830529868695860712""","""../../../../data/SampleHDS.jso…","""3382231693169699832""","""MoneyMarketSavingsAccount""","""13667322956089985814""","""Rejected"""
1.0227e19,"""4005774257469024620""","""../../../../data/SampleHDS.jso…","""2005149676010244653""","""BasicChecking""","""13667322956089985814""","""Rejected"""
1.1649e19,"""5406922016267987652""","""../../../../data/SampleHDS.jso…","""17807763110027154495""","""BasicChecking""","""9780426273716701914""","""Accepted"""
6.3281e18,"""8830529868695860712""","""../../../../data/SampleHDS.jso…","""1929638633887271205""","""UPlusFinPersonal""",null,"""Rejected"""
1.1181e19,"""8830529868695860712""","""../../../../data/SampleHDS.jso…","""559603903041144561""","""BasicChecking""","""9780426273716701914""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
8.2149e18,"""3305768747399591814""","""../../../../data/SampleHDS.jso…","""17048238929157490858""","""FirstMortgage30yr""","""1632972459100486698""","""Rejected"""
8.2271e18,"""7163918921050708176""","""../../../../data/SampleHDS.jso…","""457546386946707668""","""FirstMortgage30yr""","""11038258803662050144""","""Accepted"""
8.4128e17,"""1475552155946704095""","""../../../../data/SampleHDS.jso…","""4893318028480005097""","""MoneyMarketSavingsAccount""","""1632972459100486698""","""Rejected"""
1.6675e19,"""7163918921050708176""","""../../../../data/SampleHDS.jso…","""667546878277192162""","""BasicChecking""","""1632972459100486698""","""Rejected"""
1.5882e19,"""3305768747399591814""","""../../../../data/SampleHDS.jso…","""18309624764761896219""","""BasicChecking""","""1607738534331524194""","""Accepted"""
1.6991e19,"""1475552155946704095""","""../../../../data/SampleHDS.jso…","""9923310107525205779""","""UPlusFinPersonal""",null,"""Rejected"""
1.3015e19,"""1475552155946704095""","""../../../../data/SampleHDS.jso…","""6102050236977356705""","""BasicChecking""","""1607738534331524194""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
filename=filename
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2374e19,"""8143261962905105602""","""9334090387956600128""","""FirstMortgage30yr""","""17559360017661004494""","""Rejected"""
3.3676e17,"""17712886497653253379""","""545985962002756100""","""FirstMortgage30yr""","""15540437949117486736""","""Accepted"""
8.5590e17,"""16358691121446902796""","""13362896636285626020""","""MoneyMarketSavingsAccount""","""17559360017661004494""","""Rejected"""
1.6751e18,"""17712886497653253379""","""16069116419132877782""","""BasicChecking""","""17559360017661004494""","""Rejected"""
2.4591e18,"""8143261962905105602""","""12207439731929228649""","""BasicChecking""","""10514018318995413814""","""Accepted"""
1.3862e19,"""16358691121446902796""","""2891111801820457223""","""UPlusFinPersonal""",null,"""Rejected"""
1.5625e19,"""16358691121446902796""","""14807234862678069604""","""BasicChecking""","""10514018318995413814""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'